<a href="https://colab.research.google.com/github/Satya-Mohan/Condition-control-statements/blob/main/AI_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import keras
import tensorflow as tf
from keras import layers
import numpy as np
from tensorflow.keras.preprocessing import text_dataset_from_directory
import re
import string
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import LeakyReLU

In [64]:
#file path
base_path ='/content/drive/MyDrive/Satya_aclImdb'
imdb_path_train='/train'
imdb_path_test='/test'

In [65]:
full_path = base_path+imdb_path_train
batch_size = 32

In [66]:
# Creating a tensorFlow dataset for the training testing and validation sets
full_path = base_path+imdb_path_train
df_imdb_train = text_dataset_from_directory(full_path,batch_size= batch_size, validation_split=0.2, subset='training', seed= 42 )
df_imdb_val = text_dataset_from_directory(full_path,batch_size= batch_size, validation_split=0.2, subset='validation', seed= 42 )
full_path = base_path+imdb_path_test
df_imdb_test = text_dataset_from_directory(full_path,batch_size= batch_size)

Found 25147 files belonging to 2 classes.
Using 20118 files for training.
Found 25147 files belonging to 2 classes.
Using 5029 files for validation.
Found 25070 files belonging to 2 classes.


In [67]:
print(type(df_imdb_train))
print(df_imdb_train.take(1))
print(f"Number of batches in df_imdb_train: {df_imdb_train.cardinality()}")
print(f"Number of batches in df_imdb_val: {df_imdb_val.cardinality()}")
print(f"Number of batches in df_imdb_test: {df_imdb_test.cardinality()}")

<class 'tensorflow.python.data.ops.batch_op._BatchDataset'>
<_TakeDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Number of batches in df_imdb_train: 629
Number of batches in df_imdb_val: 158
Number of batches in df_imdb_test: 784


In [68]:
for text_batch, label_batch in df_imdb_train.take(1):
    for i in range(10):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b"Released in 1965, but clearly shot years earlier, this is an inept little crime melodrama with some inept sexploitation up front. As usual for grindhouse flicks of era, there's a fair amount of undressing and dressing for no reason complemented by lousy music, annoying narration, and awkward editing. The coffee shop scene lays the excruciating groundwork, as we chop back and forth between characters to avoid actually seeing them speak their lines. All we get are reaction shots to the off-screen character's voice! 50s-pretty Misty Ayers strips to her French-cut panties a couple of times before the action gets started. She's accompanied continuously by what is apparently stock music from romantic to western to mother-does-the-dishes, mixed randomly to produce, among other things, the most thrilling cigarette lighting ever captured on film. Watch as he taps it! Watch as he strikes the match! Will he inhale or will he be captured by Apaches? Only time will tell!! The film tells the sordi

In [74]:
def custom_standarize_data(data):
    data_filtered = tf.strings.lower(data)
    data_filtered = tf.strings.regex_replace(data_filtered, "<br />", " ")
    data_filtered = tf.strings.regex_replace(data_filtered, f"[{re.escape(string.punctuation)}]", "")
    return data_filtered

In [75]:
max_features = 15000

unigram_vectorizer = TextVectorization(
    standardize=custom_standarize_data,
    max_tokens=max_features,
    output_mode='tf-idf',
    ngrams=1,
    #output_sequence_length=sequence_length)# can ony be used when output_mode = int
    )

In [76]:
# Bigram model
bigram_vectorizer = TextVectorization(
    standardize=custom_standarize_data,
    max_tokens=max_features,
    output_mode='tf-idf',
    ngrams=2,
    #output_sequence_length=sequence_length)
    )

text_ds = df_imdb_train.map(lambda x, y: x)
unigram_vectorizer.adapt(text_ds)
bigram_vectorizer.adapt(text_ds)

In [77]:
# Create a bag-of-words model with unigrams
unigram_model = tf.keras.models.Sequential([
    unigram_vectorizer,
    tf.keras.layers.Dense(52, activation=LeakyReLU(alpha=0.1)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [78]:
# Compile and fit the bag-of-words model with unigrams
unigram_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
unigram_model.fit(df_imdb_train, validation_data=df_imdb_val, epochs=10)

loss, accuracy = unigram_model.evaluate(df_imdb_test)

print('Unigram model performance:')
print("Test set accuracy: {:.2f}".format(accuracy))
loss, accuracy = unigram_model.evaluate(df_imdb_test)
print("Test set accuracy: {:.2f}".format(accuracy))
# Create a bag-of-words model with bigrams
bigram_model = tf.keras.models.Sequential([
    bigram_vectorizer,
    tf.keras.layers.Dense(52, activation=LeakyReLU(alpha=0.1)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Epoch 1/10
629/629 [==============================] - 42s 64ms/step - loss: 0.3489 - accuracy: 0.8553 - val_loss: 0.2671 - val_accuracy: 0.8920
Epoch 2/10
629/629 [==============================] - 36s 57ms/step - loss: 0.1511 - accuracy: 0.9443 - val_loss: 0.3152 - val_accuracy: 0.8843
Epoch 3/10
629/629 [==============================] - 35s 56ms/step - loss: 0.0785 - accuracy: 0.9734 - val_loss: 0.3697 - val_accuracy: 0.8793
Epoch 4/10
629/629 [==============================] - 41s 65ms/step - loss: 0.0437 - accuracy: 0.9866 - val_loss: 0.4561 - val_accuracy: 0.8787
Epoch 5/10
629/629 [==============================] - 37s 58ms/step - loss: 0.0335 - accuracy: 0.9899 - val_loss: 0.4877 - val_accuracy: 0.8767
Epoch 6/10
629/629 [==============================] - 36s 57ms/step - loss: 0.0305 - accuracy: 0.9902 - val_loss: 0.5303 - val_accuracy: 0.8759
Epoch 7/10
629/629 [==============================] - 36s 57ms/step - loss: 0.0186 - accuracy: 0.9942 - val_loss: 0.6514 - val_accuracy:

In [80]:
print(type(df_imdb_test))

<class 'tensorflow.python.data.ops.batch_op._BatchDataset'>


In [81]:
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>

SyntaxError: ignored

In [ ]:
#Sequential model with LSTM and embedding layer

In [ ]:
import keras
import tensorflow as tf
from keras import layers

In [ ]:
#Running this line should output “Found 20000 files belonging to 2 classes”
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
"path/train", batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
"path/test", batch_size=batch_size
)


In [ ]:
max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
max_tokens=max_tokens,
output_mode="int",
output_sequence_length=max_length,
)

In [ ]:
text_only_train_ds = train_ds.map(lambda x, y: x) # Prepare a dataset that only yields raw text inputs (no labels)

In [ ]:
text_vectorization.adapt(text_only_train_ds) #Use that dataset to index the dataset vocabulary via the adapt() method

In [ ]:
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64") #This creates a Keras input layer that can accept sequences of integers with variable length (shape=(None,))

In [ ]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256) 
#input_dim is the maximum number of tokens (words) in the vocabulary
#output_dim is the size of the embedding vector - for each word in the vocabulary, a vector of size 256 is learned by the embedding layer

In [ ]:
max_length = 600

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.LSTM(32)(embedded)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer="rmsprop",
loss="binary_crossentropy",
metrics=["accuracy"])

In [ ]:
model.fit(int_train_ds, validation_data=int_test_ds, epochs=10)

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers

In [ ]:
# Toy training data
texts = ["positive text", "negative text", "neutral text", "positive review", "negative review"]
labels = [1, 0, 0, 1, 0]

# Text vectorization layer
max_tokens = 100
text_vectorization = layers.TextVectorization(max_tokens=max_tokens)
text_vectorization.adapt(texts)

# Define the model
embedding_dim = 16
model = keras.Sequential([
    text_vectorization,
    layers.Embedding(input_dim=max_tokens, output_dim=embedding_dim), #new thing
    layers.LSTM(32),
    layers.Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
batch_size = 2
epochs = 10
history = model.fit(np.array(texts), np.array(labels), batch_size=batch_size, epochs=epochs)


In [ ]:
Epoch 1/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0169 - accuracy: 0.9948 - val_loss: 1.0917 - val_accuracy: 0.8642
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0097 - accuracy: 0.9972 - val_loss: 1.0315 - val_accuracy: 0.8690
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0128 - accuracy: 0.9970 - val_loss: 1.0379 - val_accuracy: 0.8714
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 1.1487 - val_accuracy: 0.8704
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 6.7784e-04 - accuracy: 0.9998 - val_loss: 1.2389 - val_accuracy: 0.8728
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0214 - accuracy: 0.9962 - val_loss: 1.3027 - val_accuracy: 0.8524
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0162 - accuracy: 0.9958 - val_loss: 0.8921 - val_accuracy: 0.8728
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 1.1458 - val_accuracy: 0.8706
Epoch 9/10
625/625 [==============================] - 3s 5ms/step - loss: 3.8451e-04 - accuracy: 0.9999 - val_loss: 1.2083 - val_accuracy: 0.8746
Epoch 10/10
625/625 [==============================] - 3s 5ms/step - loss: 6.9276e-05 - accuracy: 1.0000 - val_loss: 1.2669 - val_accuracy: 0.8730
Sequential model performance:
782/782 [==============================] - 3s 3ms/step - loss: 1.3422 - accuracy: 0.8699